In [2]:
import tarfile
import gzip
import os
import nltk 
import pandas as pd
from nltk.tokenize import word_tokenize   
from nltk.corpus import stopwords         
from nltk.stem import WordNetLemmatizer   
from nltk.stem import PorterStemmer      
import string
from tqdm import tqdm
from glob import glob
import re

In [2]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package punkt to C:\Users\Arya
[nltk_data]     Yu\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to C:\Users\Arya
[nltk_data]     Yu\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Arya Yu\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to C:\Users\Arya
[nltk_data]     Yu\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to C:\Users\Arya
[nltk_data]     Yu\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [13]:
def text_process(text):
    res = []
    if text:
        text = text.strip('RT')
        at_pattern = re.compile("@[\S]*")
        url_pattern = re.compile("https:\/\/[\w./]*[\w/]")
        text = re.sub(at_pattern,'',text)
        text = re.sub(url_pattern,'',text)
        tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
        tokens = tokenizer.tokenize(text.lower())
        stop_words = set(stopwords.words('english'))
        stop_words.update(list(string.punctuation))
        filtered = [w for w in tokens if w not in stop_words]
        lemmatizaer = WordNetLemmatizer()
        for word, tag in nltk.pos_tag(filtered):
            if tag.startswith('NN'):
                word= lemmatizaer.lemmatize(word, pos='n')
            elif tag.startswith('VB'):
                word= lemmatizaer.lemmatize(word, pos='v')
            elif tag.startswith('JJ'):
                word= lemmatizaer.lemmatize(word, pos='a')
            elif tag.startswith('R'):
                word= lemmatizaer.lemmatize(word, pos='r')
            res.append(word)
    #porter_stemmer = PorterStemmer()
    #for word in lemmatized_word:
        #res.append(porter_stemmer.stem(word))
    return res

In [ ]:
data_root = "./tweets/"
process_root = "./tweets_process/"
if not os.path.exists(process_root):
    os.makedirs(process_root)
    
def csv_process(file):
    df = pd.read_csv(file)
    df['text_process'] = df['full_text'].apply(lambda x: " ".join(text_process(x)))
    df = df.loc[:,['id','text_process']]
    df.to_csv(file.replace("tweets","tweets_process"),index=False)

for root, dirs, files in os.walk(data_root):
    for i in tqdm(range(len(files))):
        file = files[i]
        file_path = os.path.join(root,file)
        if file not in os.listdir(root.replace("tweets","tweets_process")):
            csv_process(file_path)

In [68]:
import csv
process_dict={}
for root,dirs,files in os.walk("./tweets"):
    for i in tqdm(range(len(files))):
        file = files[i]
        file_path = os.path.join(root,file)
        with open(file_path,'r',encoding='UTF-8') as csvfile:
            csv_reader = csv.reader(csvfile) 
            header = next(csv_reader)  
            for row in csv_reader:  
                if len(row)==3:
                    id = row[0]
                    text = row[2]
                    if id not in process_dict:
                        process_dict[id] = " ".join(text_process(text))       

0it [00:00, ?it/s]
100%|██████████████████████████████████████████████████████████████████████████████| 6567/6567 [32:28<00:00,  3.37it/s]


In [69]:
len(process_dict)

1136261

In [70]:
with open('data_processing.csv', 'w',newline = "",encoding='UTF-8') as csv_file:  
    writer = csv.writer(csv_file)
    writer.writerow(["id","text_process"])
    for key, value in process_dict.items():
        writer.writerow([key,value])

In [3]:
data = pd.read_csv('data_processing.csv')
process_root = "./tweets_process/"
if not os.path.exists(process_root):
    os.makedirs(process_root)

row_num = int(len(data)/10)
for i in range(0, 10):
    if i==9:
        save_data = data.iloc[i*row_num :]
    else:
        save_data = data.iloc[i*row_num : (i+1)*row_num]
    file_name = process_root + str(i) + '.csv'  
    save_data.to_csv(file_name)  